In [1]:
import os

from seml import get_results
import matplotlib.pyplot as plt
import torch
import pandas as pd

import seaborn as sns

In [ ]:
df = get_results('dp_timeseries_eval_pld_monotonicity_composed',
                 fields=['config.privacy_loss_kwargs.min_sequence_length',
                         'config.privacy_loss_kwargs.batch_size',
                         'config.privacy_loss_kwargs.instances_per_sequence',
                         'result.log_dir'],
                 filter_dict={
                    'config.privacy_loss_kwargs.top_level_mode': 'sampling_without_replacement',
                    'config.privacy_loss_kwargs.bottom_level_mode': 'sampling_with_replacement',
                    'config.privacy_loss_kwargs.future_length': 1,
                    'config.num_compositions': 1000
                },
            to_data_frame=True)

df = df.drop(columns=['_id'])

df

In [3]:
columns = df.columns

df = df.rename(columns={
    c: c.split('.')[-1]
    for c in columns
})

In [4]:
def epsilons_deltas_from_log_dir(df):
    data_dict = {
        'epsilons': [],
        'deltas': []}

    for log_dir in df['log_dir']:
        
        save_dict = torch.load(os.path.join(log_dir, 'config_and_results.pyt'))
        data_dict['epsilons'].append(save_dict['results']['epsilons'])
        data_dict['deltas'].append(save_dict['results']['deltas'])
    
    return pd.DataFrame(data_dict)

In [ ]:
df_results = pd.concat((df, epsilons_deltas_from_log_dir(df)), axis=1)
df_results.head()

In [6]:
def plot_tradeoff(df, min_sequence_length, batch_size, num_compostions, num_instances_filter=None, xlim=None):
    df = df.copy()

    df = df.loc[(df['min_sequence_length'] == min_sequence_length)
                & (df['batch_size'] == batch_size)]
    
    if num_instances_filter is not None:
        df = df.loc(df['instances_per_sequence'].isin(num_instances_filter))

    df = df[['instances_per_sequence', 'epsilons', 'deltas']]
    df = df.sort_values('instances_per_sequence')

    sns.set_theme()
    pal = sns.color_palette('colorblind', 5)

    fig, ax = plt.subplots(1, 1)
    
    for i, (_, (num_instances, epsilons, deltas)) in enumerate(df.iterrows()):

        #deltas = deltas[(0 <= epsilons) & (epsilons <= 1)]
        #epsilons = epsilons[(0 <= epsilons) & (epsilons <= 1)]
        deltas = deltas[num_compostions - 1]

        if xlim is not None:
            deltas = deltas[epsilons <= xlim]
            epsilons = epsilons[epsilons <= xlim]
        
        linestyle = 'solid' if (num_instances == 1) else 'dashed'
        ax.plot(epsilons, deltas, linestyle=linestyle, zorder=3, clip_on=False, color=pal[i])
        # Dummy lines for legend
        ax.plot([], [], label=f'{num_instances}', linestyle='solid', zorder=3, clip_on=False, color=pal[i])

    if xlim is not None:
        ax.set_xlim(left=epsilons.min(), right=xlim)
    else:
        ax.set_xlim(left=epsilons.min(), right=epsilons.max())
    ax.set_ylim(bottom=0)
    ax.set_xscale('log')
    #plt.yscale('log')

    ax.minorticks_off()

    if num_compostions == 1000:
        loc = 'upper left'
    else:
        loc = 'upper right'
    legend_1 = ax.legend(title='$\lambda$', title_fontsize=10, loc=loc)
    ax.set_xlabel('$\epsilon$')
    ax.set_ylabel('$\delta(\epsilon)$')

    linestyles = ['solid', 'dashed']
    labels = ['Tight upper bound', 'Lower bound']
    dummy_lines = []
    for linestyle, label in zip(linestyles, labels):
        dummy_lines.append(ax.plot([],[], c="black", linestyle=linestyle)[0])
    ax.legend(dummy_lines, labels, loc='lower left')

    # Add old legend back
    ax.add_artist(legend_1)

In [7]:
min_sequence_lengths = [4, 4, 4, 20, 20, 20, 20, 20, 20]
batch_sizes = [32, 32, 32, 160, 160, 160, 32, 32, 32]
num_compositions = [1, 100, 1000, 1, 100, 1000, 1, 100, 1000]

save_dir = '/ceph/hdd/staff/schuchaj/dp_timeseries_plots_camera_icml25/eval_pld_monotonicity_composed'

for min_sequence_length, batch_size, num_composition in zip(min_sequence_lengths,
                                                             batch_sizes,
                                                             num_compositions):

    plot_tradeoff(df_results, min_sequence_length, batch_size, num_composition, None)

In [8]:
min_sequence_lengths = [20, 20]
batch_sizes = [32, 32]
num_compositions = [1, 100]

save_dir = '/ceph/hdd/staff/schuchaj/dp_timeseries_plots/eval_pld_monotonicity_composed'

for min_sequence_length, batch_size, num_composition in zip(min_sequence_lengths,
                                                             batch_sizes,
                                                             num_compositions):

    plot_tradeoff(df_results, min_sequence_length, batch_size, num_composition, None, xlim=10)